In [ ]:
import numpy as np
import gc
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import warnings;warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
Parquet is an open source file format built to handle flat columnar storage data formats. 

Parquet works great with large, complex data and is known for its data compression and ability many encoding types.

Data found: https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/data

size: 986.46 MB

In [ ]:
train_parquet_path = "train_series.parquet"
test_parquet_path = "test_series.parquet"
train_events_path = "train_events.csv"
train_series = pd.read_parquet(train_parquet_path)
test_series = pd.read_parquet(test_parquet_path)
train_events = pd.read_csv(train_events_path)

In [ ]:
# removing known NANs,known incomplete series:31011ade7c0a,a596ad0b82aa
series_NaN = train_events.groupby('series_id')['step'].apply(lambda x: x.isnull().any())
non_NaN_series = series_NaN[~series_NaN].index.tolist()
non_NaN_series.remove('31011ade7c0a')
non_NaN_series.remove('a596ad0b82aa') 

In [ ]:
def parquet_genny(series):
    train_series = pd.read_parquet("train_series.parquet", filters=[('series_id','=',series)])
    train_events = pd.read_csv("train_events.csv").query('series_id == @series')
    train_events = train_events.dropna()
    train_events["step"]  = train_events["step"].astype("int")
    train_events["awake"] = train_events["event"].replace({"onset":1,"wakeup":0})
    train = pd.merge(train_series, train_events[['step','awake']], on='step', how='left')
    train["awake"] = train["awake"].bfill(axis ='rows')
    train['awake'] = train['awake'].fillna(1)  
    train["awake"] = train["awake"].astype("int")
    return(train)
parquet_data = []
for series_id in non_NaN_series:
    train = parquet_genny(series_id)
    parquet_data.append(train)
    del train
    gc.collect(); #memory help !

In [ ]:
Zzzs_train = pd.concat(parquet_data).reset_index(drop=True)
print(Zzzs_train["series_id"].nunique(), "indivudual sleep training series")

In [ ]:
#%reset -f
#file_path = 'clean_zzz.csv'
#Zzzs_train.to_csv(file_path, index=False)
#zzz = pd.read_csv(file_path)
#clear mem - 

In [ ]:
# creating possible time covariates for model 
start_time = datetime.strptime(zzz['timestamp'].iloc[0], '%Y-%m-%dT%H:%M:%S%z')
zzz['seconds_since_start'] = zzz['timestamp'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%dT%H:%M:%S%z') - start_time).seconds)
zzz = zzz.drop(columns=['timestamp', 'step'])

In [ ]:
# sorte by time series_id & seconds_since_start
zzz_timesort = zzz.sort_values(by=['series_id', 'seconds_since_start'])

Z-angle: corresponds to the angle between the accelerometer axis perpendicular to the skin surface and the horizontal plane.

ENMO : The Euclidean Norm Minus One (ENMO) with negative values rounded to zero in g has been shown to correlate with the magnitude of acceleration and human energy expenditure16. ENMO is computed as follows:

$ \text{ENMO} = \sqrt{x^2 + y^2 + z^2} - 1 $

In [ ]:
# Standardize feats: time(seconds since session start),anglez & enmo(Euclidean Norm Minus One)
zzz_timesort['anglez_std'] = (zzz_timesort['anglez'] - zzz_timesort['anglez'].mean()) / zzz_timesort['anglez'].std()
zzz_timesort['enmo_std'] = (zzz_timesort['enmo'] - zzz_timesort['enmo'].mean()) / zzz_timesort['enmo'].std()
zzz_timesort['sss_std'] = (zzz_timesort['seconds_since_start'] - zzz_timesort['seconds_since_start'].mean()) / zzz_timesort['seconds_since_start'].std()

zzz_timesort['anglez_raw'] = (zzz_timesort['anglez'])
zzz_timesort['enmo_raw'] = (zzz_timesort['enmo'])
zzz_timesort['sss_raw'] = (zzz_timesort['seconds_since_start'])


feats = [col for col in zzz_timesort.columns if 'std' in col]
X = zzz_timesort[feats]
y = zzz_timesort['awake']

feats_raw = [col for col in zzz_timesort.columns if 'raw' in col]
X_raw = zzz_timesort[feats_raw]

# split the standardized data (79-21)
train_size = int(0.79 * len(zzz_timesort))
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# split the raw training & test data (79-21)
X_tr, X_te = X_raw.iloc[:train_size], X_raw.iloc[train_size:]


In [ ]:
#file_path1 = 'X1.csv'
#file_path2 = 'y1.csv'
#file_path3 = 'X2.csv'
#file_path4 = 'y2.csv'
#X_train.to_csv(file_path1, index=False)
#y_train.to_csv(file_path2, index=False)
#X_test.to_csv(file_path3, index=False)
#y_test.to_csv(file_path4, index=False)

#file_path5 = 'X1r.csv'
#file_path6 = 'X2r.csv'
#X_tr.to_csv(file_path5, index=False)
#X_te.to_csv(file_path6, index=False)


In [ ]:
#clear mem - 
%reset -f

In [1]:
import numpy as np
import pandas as pd
import warnings;warnings.simplefilter(action='ignore', category=Warning)
file_path1 = 'X1.csv'
file_path2 = 'y1.csv'
file_path3 = 'X2.csv'
file_path4 = 'y2.csv'
X1 = pd.read_csv(file_path1)
y1 = pd.read_csv(file_path2)
X2 = pd.read_csv(file_path3)
y2 = pd.read_csv(file_path4)

X_train = np.array(X1);X_test = np.array(X2)
y_train = np.array(y1).flatten();y_test = np.array(y2).flatten()

In [2]:
# numpy only data peek

def data_info(data):
    missing_values = np.isnan(data).sum()
    size_in_mb = data.nbytes / (1024 ** 2)
    if len(data.shape) > 1:
        mean = data.mean()
        sd = data.std()
    else:
        mean = np.mean(data)
        sd = np.std(data)
    return missing_values, size_in_mb, mean, sd

def data_summary(X_train, y_train, X_test, y_test):
    header = "+------------+-------------------------+------------------+--------------+---------+-----------+------------+"
    print(header)
    print("| Data       | Type                    | Shape            | MissingVals  | Size(MB)| Mean      | SD         |")
    print(header)

    for name, data in [("X_train", X_train), ("y_train", y_train), ("X_test", X_test), ("y_test", y_test)]:
        missing_values, size_in_mb, mean, sd = data_info(data)
        print("| {:<9} | {:<24} | {:<16} | {:<12} | {:<7.2f} | {:<9.2f} | {:<10.2f} |".format(
            name, str(type(data)), str(data.shape), missing_values, size_in_mb, mean, sd))
    print(header)


data_summary(X_train, y_train, X_test, y_test)

+------------+-------------------------+------------------+--------------+---------+-----------+------------+
| Data       | Type                    | Shape            | MissingVals  | Size(MB)| Mean      | SD         |
+------------+-------------------------+------------------+--------------+---------+-----------+------------+
| X_train   | <class 'numpy.ndarray'>  | (10400792, 3)    | 0            | 238.06  | -0.00     | 0.99       |
| y_train   | <class 'numpy.ndarray'>  | (10400792,)      | 0            | 79.35   | 0.66      | 0.47       |
| X_test    | <class 'numpy.ndarray'>  | (2764768, 3)     | 0            | 63.28   | 0.01      | 1.04       |
| y_test    | <class 'numpy.ndarray'>  | (2764768,)       | 0            | 21.09   | 0.64      | 0.48       |
+------------+-------------------------+------------------+--------------+---------+-----------+------------+


Covariates standardized:

column 1: Z-angle, corresponds to the angle between the accelerometer axis perpendicular to the skin surface and the horizontal plane.

${angl}{{e}}_{{z}}=({ta}{{n}}^{-1}\frac{{{a}}_{{z}}}{{{a}}_{x}^{2}+{{a}}_{y}^{2}})\cdot 180/\pi ,$

column 2: ENMO, The Euclidean Norm Minus One with negative values rounded to zero in g has been shown to correlate with the magnitude of acceleration and human energy expenditure16. ENMO is computed as follows:

$ \text{ENMO} = \sqrt{x^2 + y^2 + z^2} - 1 $

column3: sss, seconds since session start 

In [8]:
# Metric Stuff

def confusion_matrix(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    return TP, TN, FP, FN

def accuracy(TP, TN, FP, FN):
    return (TP + TN) / (TP + TN + FP + FN)

def specificity(TN, FP):
    return TN / (TN + FP)

def sensitivity(TP, FN):
    return TP / (TP + FN)

def f1_score(TP, FP, FN):
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return 2 * (precision * recall) / (precision + recall)

def print_confusion_matrix(TP, TN, FP, FN):
    head_cm = "+-----------------+------------+------+"
    print(head_cm)
    print("|                 | Predicted        |")
    print("| Actual          |      0    |   1  |")
    print(head_cm)
    print("| 0               | {:<5} | {:<5}   |".format(TN, FP))
    print("| 1               | {:<5} | {:<5}  |".format(FN, TP))
    print(head_cm)
    print()

def metrics(y_true, y_pred):
    TP, TN, FP, FN = confusion_matrix(y_true, y_pred)
    acc = accuracy(TP, TN, FP, FN)
    spec = specificity(TN, FP)
    sens = sensitivity(TP, FN)
    f1 = f1_score(TP, FP, FN)

    print_confusion_matrix(TP, TN, FP, FN)
    head = "+--------------+------------+"
    print(head)
    print("| Metric       | Value      |")
    print(head)
    print("| Accuracy     | {:10.4f} |".format(acc))
    print("| Specificity  | {:10.4f} |".format(spec))
    print("| Sensitivity  | {:10.4f} |".format(sens))
    print("| F1 Score     | {:10.4f} |".format(f1))
    print(head)

def linear_mod(bias, theta):
    terms = [
    f"{bias:.8f} * bias(intercept)",
    f"{theta[0]:.8f} * angleZ",
    f"{theta[1]:.8f} * EuclideanNormMinusOne",
    f"{theta[2]:.8f} * SecondsSinceStart" ]
    return " + ".join(terms)

Logistic regression models the probability that the target variable belongs to a particular category. 

It uses the logistic function to squeeze the output of a linear equation between 0 and 1.

Given the weights
$\mathbf{w}$ and input $\mathbf{x}$, the logistic function is:

$P(y=1|\mathbf{x}) = \frac{1}{1 + e^{-\mathbf{w}^T \mathbf{x}}}$

To fit a logistic regression model while restricted to numpy, I used gradient descent.

Keep in mind a logistic regression is truely a classification, so the metrics are adjusted to such.

The gradient of the logistic loss with respect to the weights is:

$\frac{\partial L}{\partial \mathbf{w}} = (\mathbf{y} - \hat{\mathbf{y}}) \mathbf{x}$

Where:
$  L $ is the logistic loss.
$\mathbf{y}$ is the true label.
$\hat{\mathbf{y}}$ is the predicted probability.

In [9]:
# Binary target: logisitic regression which is acutally classificaiton.

class LogisticRegression:

    def __init__(self, lr, iter):
        self.lr = lr
        self.iter = iter

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # initialization of weights
        self.theta = np.zeros(X.shape[1])
        self.bias = 0

        for i in range(self.iter):
            model = np.dot(X, self.theta) + self.bias
            predictions = self.sigmoid(model)

            # compute gradient
            d_theta = (1 / len(X)) * np.dot(X.T, (predictions - y))
            d_bias = (1 / len(X)) * np.sum(predictions - y)

            # update my weights
            self.theta -= self.lr * d_theta
            self.bias -= self.lr * d_bias

    def predict_prob(self, X):
        return self.sigmoid(np.dot(X, self.theta) + self.bias)

    def predict(self, X, threshold=0.500):
        return (self.predict_prob(X) >= threshold).astype(int)



In [12]:
learning_rates = [0.25, 0.1, 0.01, 0.001,0.000001]

for lr in learning_rates:
    model_logreg = LogisticRegression(lr=lr, iter=1000)
    model_logreg.fit(X_train, y_train)
    y_pred_logreg = model_logreg.predict(X_test)
    print("learning rate",lr)
    print(metrics(y_test,y_pred_logreg))
    model_str = linear_mod(model_logreg.bias, model_logreg.theta)
    print(model_str)

learning rate 0.25
+-----------------+------------+------+
|                 | Predicted        |
| Actual          |      0    |   1  |
+-----------------+------------+------+
| 0               | 811402 | 170937   |
| 1               | 250730 | 1531699  |
+-----------------+------------+------+

+--------------+------------+
| Metric       | Value      |
+--------------+------------+
| Accuracy     |     0.8475 |
| Specificity  |     0.8260 |
| Sensitivity  |     0.8593 |
| F1 Score     |     0.8790 |
+--------------+------------+
None
1.80955572 * bias(intercept) + 0.16049870 * angleZ + 3.90823003 * EuclideanNormMinusOne + -1.51501577 * SecondsSinceStart
learning rate 0.1
+-----------------+------------+------+
|                 | Predicted        |
| Actual          |      0    |   1  |
+-----------------+------------+------+
| 0               | 785927 | 196412   |
| 1               | 259182 | 1523247  |
+-----------------+------------+------+

+--------------+------------+
| Metric

Even though I will not persue a linear model in this manner, I successfully reached 84% accuracy with a 0.25 learning rate.

Linear model found: 

1.80955572 * bias(intercept) + 0.16049870 * angleZ + 3.90823003 * EuclideanNormMinusOne + -1.51501577 * SecondsSinceStart

